## I dont know type of shit???
Mobilenet and sumthing else i guess

In [1]:
# Changing directory to access the database
import os
os.chdir("/Users/komangandikawirasantosa/Face-Rec/DeepFace-IPYNB")

In [2]:
import os
import cv2
import dlib
import numpy as np
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from keras.models import Model
from sklearn.preprocessing import Normalizer
import matplotlib.pyplot as plt

In [3]:
detector = dlib.get_frontal_face_detector()

In [4]:
# Load the MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')
model = Model(inputs=base_model.input, outputs=base_model.output)

/var/folders/n5/zv_kb44d377fm2dm5gts2dwm0000gn/T/ipykernel_84583/199141259.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')


In [5]:
def get_face_embedding(image):
    # Detect faces in the image
    detections = detector(image, 1)
    embeddings = []
    
    for d in detections:
        # Extract the face region
        face = image[d.top():d.bottom(), d.left():d.right()]
        # Resize the face to the input size of the model
        face = cv2.resize(face, (224, 224))
        face = np.expand_dims(face, axis=0)
        # Preprocess the face
        face = preprocess_input(face)
        # Get the embedding
        embedding = model.predict(face)
        embeddings.append(embedding.flatten())
    
    return embeddings


In [6]:
def create_embeddings(data_dir):
    embeddings = {}
    for person_name in os.listdir(data_dir):
        person_dir = os.path.join(data_dir, person_name)
        if os.path.isdir(person_dir):
            embeddings[person_name] = []
            for image_name in os.listdir(person_dir):
                image_path = os.path.join(person_dir, image_name)
                image = cv2.imread(image_path)
                
                # Check if the image was loaded successfully
                if image is None:
                    print(f"Failed to load image: {image_path}")
                    continue  # Skip this image if it failed to load
                
                face_embeddings = get_face_embedding(image)
                if face_embeddings:
                    embeddings[person_name].append(face_embeddings[0])
    return embeddings

In [7]:
def normalize_embeddings(embeddings):
    normalized_embeddings = {}
    for name, encodings in embeddings.items():
        norm = Normalizer()
        normalized_embeddings[name] = norm.fit_transform(encodings)
    return normalized_embeddings


In [18]:
import cv2
import numpy as np

def cosine_similarity(a, b):
    """Calculate the cosine similarity between two vectors."""
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    if norm_a == 0 or norm_b == 0:
        return 0
    return dot_product / (norm_a * norm_b)

def recognize_faces(test_image_path, embeddings, threshold=0.45):
    test_image = cv2.imread(test_image_path)
    rgb_test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)
    
    # Detect faces in the test image
    face_locations = detector(rgb_test_image, 1)
    face_encodings = get_face_embedding(rgb_test_image)
    
    # Initialize a list for recognized names and distances
    recognized_names = []
    distances = []
    
    for face_encoding in face_encodings:
        # Compare the face with the known embeddings
        similarities = []
        for name, known_encodings in embeddings.items():
            for known_encoding in known_encodings:
                similarity = cosine_similarity(face_encoding, known_encoding)
                similarities.append((name, similarity))
        
        # Find the closest match
        if similarities:
            name, similarity = max(similarities, key=lambda x: x[1])
            distance = 1 - similarity  # Calculate distance from similarity
            
            # Check if the similarity is above the threshold
            if similarity > threshold:
                recognized_names.append(name)
            else:
                recognized_names.append("Unknown")
            
            distances.append(distance)  # Append the distance score
        else:
            recognized_names.append("Unknown")
            distances.append(None)  # No distance available for unknowns
    
    return test_image, face_locations, recognized_names, distances

In [9]:
def draw_results(image, face_locations, names, distances):
    for (d, name, distance) in zip(face_locations, names, distances):
        top, right, bottom, left = d.top(), d.right(), d.bottom(), d.left()
        cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)
        text = f"{name} ({distance:.2f})" if distance is not None else "Unknown"
        cv2.putText(image, text, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    return image

def save_result(image, output_path):
    cv2.imwrite(output_path, image)

In [10]:
# Directory where the images are stored
data_dir = 'cropped_faces'  # Update this path
# Create embeddings
embeddings = create_embeddings(data_dir)
normalized_embeddings = normalize_embeddings(embeddings)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 625ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━

In [19]:
# Path to the test image
test_image_path = 'Test1.jpeg'  # Update this path

# Recognize faces in the test image
test_image, face_locations, recognized_names, distances = recognize_faces(test_image_path, normalized_embeddings)

# Draw results
result_image = draw_results(test_image, face_locations, recognized_names, distances)

# Save the result image
output_path = 'output(dlib+mobilenet)/output1.jpg'  # Specify the output path
os.makedirs(os.path.dirname(output_path), exist_ok=True)  # Create the output directory
save_result(result_image, output_path)

print(f"Result image saved to: {output_path}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Result image saved to: output(dlib+mobilenet)/output1.jpg


In [20]:
# Path to the test image
test_image_path = 'Test2.jpeg'  # Update this path

# Recognize faces in the test image
test_image, face_locations, recognized_names, distances = recognize_faces(test_image_path, normalized_embeddings)

# Draw results
result_image = draw_results(test_image, face_locations, recognized_names, distances)

# Save the result image
output_path = 'output(dlib+mobilenet)/output2.jpg'  # Specify the output path
os.makedirs(os.path.dirname(output_path), exist_ok=True)  # Create the output directory
save_result(result_image, output_path)

print(f"Result image saved to: {output_path}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Result image saved to: output(dlib+mobilenet)/output2.jpg


## Having existensial crisis